In [1]:
import numpy as np
import pandas as pd

import random
random.seed(28)
np.random.seed(28)


import matplotlib.pyplot as plt
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)
import os
import copy
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15
from collections import defaultdict
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import time
from collections import Counter
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import (SimpleImputer, IterativeImputer, MissingIndicator)
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import BayesianOptimization
import eli5
import shap
from IPython.display import HTML
import json

import matplotlib.pyplot as plt
%matplotlib inline
import os
import time
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
pd.set_option('max_rows', 500)
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option('display.max_columns', 1000)
np.random.seed(566)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.


# Read the data

In [2]:
train = pd.read_csv("data/training_v2.csv")
samplesubmission = pd.read_csv("data/samplesubmission.csv")
test = pd.read_csv("data/unlabeled.csv")
dictionary = pd.read_csv("data/WiDS Datathon 2020 Dictionary.csv")
solution_template = pd.read_csv("data/solution_template.csv")
df = pd.concat([train,test])

print('train ' , train.shape)
print('test ' , test.shape)
print('samplesubmission ' , samplesubmission.shape)
print('solution_template ' , solution_template.shape)
print('dictionary ' , dictionary.shape)

train  (91713, 186)
test  (39308, 186)
samplesubmission  (3, 2)
solution_template  (39308, 2)
dictionary  (188, 6)


> > # OverView of the dataset

In [3]:
dico=pd.DataFrame(dictionary.T.head(6))
dico.columns=list(dico.loc[dico.index == 'Variable Name'].unstack())
dico = dico.loc[dico.index != 'Variable Name']
dico.columns
train_stat = pd.DataFrame(train.describe())
train_stat2 = pd.concat([dico,train_stat],axis=0)
train_stat2.head(20)

,age,aids,albumin_apache,apache_2_bodysystem,apache_2_diagnosis,apache_3j_bodysystem,apache_3j_diagnosis,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,apache_post_operative,arf_apache,bilirubin_apache,bmi,bun_apache,cirrhosis,creatinine_apache,d1_albumin_max,d1_albumin_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_heartrate_max,d1_heartrate_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_resprate_max,d1_resprate_min,d1_sodium_max,d1_sodium_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,d1_wbc_max,d1_wbc_min,diabetes_mellitus,elective_surgery,encounter_id,ethnicity,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,gender,glucose_apache,h1_albumin_max,h1_albumin_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_heartrate_max,h1_heartrate_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_resprate_max,h1_resprate_min,h1_sodium_max,h1_sodium_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,h1_wbc_max,h1_wbc_min,heart_rate_apache,height,hematocrit_apache,hepatic_failure,hospital_admit_source,hospital_death,hospital_id,icu_admit_source,icu_admit_type,icu_id,icu_stay_type,icu_type,immunosuppression,intubated_apache,leukemia,lymphoma,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,patient_id,ph_apache,pre_icu_los_days,pred,readmission_status,resprate_apache,sodium_apache,solid_tumor_with_metastasis,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,weight
Category,demographic,APACHE comorbidity,APACHE covariate,APACHE grouping,APACHE covariate,APACHE grouping,APACHE covariate,APACHE prediction,APACHE prediction,APACHE covariate,APACHE covariate,APACHE covariate,demographic,APACHE covariate,APACHE comorbidity,APACHE covariate,labs,labs,labs blood gas,labs blood gas,labs blood gas,labs blood gas,labs blood gas,labs blood gas,labs,labs,labs,labs,labs,labs,labs,labs,vitals,vitals,vitals,vitals,vitals,vitals,labs,labs,labs,labs,vitals,vitals,labs,labs,labs,labs,labs,labs,labs,labs,vitals,vitals,vitals,vitals,vitals,vitals,labs blood gas,labs blood gas,labs,labs,labs,labs,vitals,vitals,labs,labs,vitals,vitals,vitals,vitals,vitals,vitals,vitals,vitals,vitals,vitals,labs,labs,APACHE comorbidity,demographic,identifier,demographic,APACHE covariate,APACHE covariate,APACHE covariate,APACHE covariate,APACHE covariate,demographic,APACHE covariate,labs,labs,labs blood gas,labs blood gas,labs blood gas,labs blood gas

# Adversarial validation - hospital_id is disjoint!
Run externally - we have perfect seperation on basis of ICU ID or hospital id
For now I add variables to check overlaps
validation set should possibly be split by icu_id or hospital id , rather than random . This also affects features we can create
There is a small subset of hospital_ids that overlap - ~ 1K rows
ICU and hospital ID are synonymous for this purpose

In [ ]:
df["icu_id_isin_test"] = df['icu_id'].isin(test['icu_id'])
test["icu_id_isin_test"] = True

test["icu_id_isin_train"] = test['icu_id'].isin(df['icu_id'])
df["icu_id_isin_train"] = True


df["hospital_id_isin_test"] = df['hospital_id'].isin(test['hospital_id'])
test["hospital_id_isin_test"] = True

test["hospital_id_isin_train"] = test['hospital_id'].isin(df['hospital_id'])
df["hospital_id_isin_train"] = True

In [ ]:
df[["hospital_id_isin_test","icu_id_isin_test"]].describe()

In [ ]:
test[["hospital_id_isin_train","icu_id_isin_train"]].describe()

In [ ]:
USELESS_COLS = ["patient_id",
                "hospital_id_isin_train","icu_id_isin_train","icu_id_isin_test",

> > # imputations

In [4]:
#functions
def cat_exploration(column):
    return train[column].value_counts()

def cat_imputation(column, value):
    train.loc[train[column].isnull(),column] = value
    test.loc[test[column].isnull(),column] = value

def show_missing(ds):
    missing = ds.columns[train.isnull().any()].tolist()
    return missing
def num_to_cat(feature, new_feature, labels,scale):
    
    train[new_feature] = pd.cut(train[feature], scale, labels=labels).values.add_categories('None')
    test[new_feature] = pd.cut(test[feature], scale, labels=labels).values.add_categories('None')
    cat_imputation(new_feature,"None")
    return

In [5]:
train[show_missing(train)].isnull().sum()

age                              4228 
bmi                              3429 
ethnicity                        1395 
gender                           25   
height                           1334 
hospital_admit_source            21409
icu_admit_source                 112  
weight                           2720 
albumin_apache                   54379
apache_2_diagnosis               1662 
apache_3j_diagnosis              1101 
arf_apache                       715  
bilirubin_apache                 58134
bun_apache                       19262
creatinine_apache                18853
fio2_apache                      70868
gcs_eyes_apache                  1901 
gcs_motor_apache                 1901 
gcs_unable_apache                1037 
gcs_verbal_apache                1901 
glucose_apache                   11036
heart_rate_apache                878  
hematocrit_apache                19878
intubated_apache                 715  
map_apache                       994  
paco2_apache             

In [6]:
test[show_missing(test)].isnull().sum()

age                              1654 
bmi                              1086 
ethnicity                        171  
gender                           15   
height                           750  
hospital_admit_source            11948
icu_admit_source                 115  
weight                           764  
albumin_apache                   24650
apache_2_diagnosis               896  
apache_3j_diagnosis              637  
arf_apache                       179  
bilirubin_apache                 25292
bun_apache                       7016 
creatinine_apache                6896 
fio2_apache                      29718
gcs_eyes_apache                  1163 
gcs_motor_apache                 1163 
gcs_unable_apache                546  
gcs_verbal_apache                1163 
glucose_apache                   4530 
heart_rate_apache                306  
hematocrit_apache                7747 
intubated_apache                 179  
map_apache                       301  
paco2_apache             

In [7]:
#imputations added by Nurit
#mean/median
cat_imputation('age',df['age'].mean())
cat_imputation('bun_apache',df['bun_apache'].median())
cat_imputation('resprate_apache',df['resprate_apache'].median())
cat_imputation('heart_rate_apache',df['heart_rate_apache'].mean())
cat_imputation('hematocrit_apache',df['hematocrit_apache'].mean())
cat_imputation('paco2_for_ph_apache',df['paco2_for_ph_apache'].median())
cat_imputation('paco2_apache',df['paco2_apache'].mean())
cat_imputation('pao2_apache',df['pao2_apache'].mean())


#freq
cat_imputation('ventilated_apache',0)
cat_imputation('aids',0.00)
cat_imputation('cirrhosis',0.00)
cat_imputation('hepatic_failure',0.00)
cat_imputation('lymphoma',0.00)
cat_imputation('leukemia',0.00)
cat_imputation('arf_apache',0.00)
cat_imputation('solid_tumor_with_metastasis',0.00)
cat_imputation('urineoutput_apache',0.00)
cat_imputation('gcs_unable_apache',0.00)

#categorial
cat_imputation('ethnicity','Other/Unknown')
cat_imputation('gender','None')
cat_imputation('hospital_admit_source','None')
cat_imputation('icu_admit_source','None')
cat_imputation('elective_surgery',2)
cat_imputation('diabetes_mellitus',2)

In [8]:
# #from number to categorial

num_to_cat('sodium_apache', 'sodium_cat',["normal", "high"],[0,140,np.inf])
num_to_cat('wbc_apache', 'wbc_cat', ["low","normal", "high"], [0,4,12,np.inf])
num_to_cat('creatinine_apache', 'creatinine_cat', ["low","normal", "high"],  [0,0.7, 1.3,np.inf])
num_to_cat('fio2_apache', 'fio2_cat',["low","normal", "high"], [0,0.5, 0.7,np.inf])
num_to_cat('temp_apache','temp_cat',["low","normal", "high"],[0,35, 39,np.inf])
num_to_cat('albumin_apache','albumin_cat',["low","normal", "high"],[0,2,3.3,np.inf])
num_to_cat('bilirubin_apache','bilirubin_cat',["normal", "high"], [0,1,np.inf])
num_to_cat('map_apache','map_cat',["low","normal", "high"],[0,50,170,np.inf])
num_to_cat('gcs_eyes_apache','gcs_eyes_cat',["low","normal1","noraml2","high"],[0,2,3,4,np.inf])
num_to_cat('gcs_motor_apache','gcs_motor_cat',["low","normal1","noraml2","high"],[0,2,3,4,np.inf])
num_to_cat('gcs_verbal_apache','gcs_verbal_cat',["low","normal","noraml2","high"],[0,2,3,4,np.inf])
num_to_cat('intubated_apache','intubated_cat', ['y','n'],[0,1,np.inf])
num_to_cat('ph_apache','ph_cat', ['low','normal', "high"],[6,7,7.4,np.inf])
num_to_cat('glucose_apache','glucose_cat' ,["vlow","low","normal", "high", "vhigh"], [0,30,90,130,160,np.inf])

In [9]:
features_to_drop = ['albumin_apache', 'bilirubin_apache','creatinine_apache',
                                                        'fio2_apache', 'temp_apache','sodium_apache',
                                                        'wbc_apache', 'map_apache', 'intubated_apache',
                                                        'glucose_apache','ph_apache',
                                                        'gcs_verbal_apache','gcs_eyes_apache','gcs_motor_apache',
                                                        #'gcs_unable_apache',#,'hematocrit_apache'
                   ]
features_to_add_categorical = ['albumin_cat','bilirubin_cat',
                               'elective_surgery','diabetes_mellitus',
                               'creatinine_cat','fio2_cat', 'temp_cat',
                         'sodium_cat', 'wbc_cat', 'map_cat','intubated_cat','glucose_cat','ph_cat',
                         'gcs_verbal_cat','gcs_eyes_cat','gcs_motor_cat'
                         #'gcs_unable_cat'#'hematocrit_cat'
                              ]

In [10]:
train['bmi'].fillna(train['weight']/((train['height']/100)**2),inplace=True)
test['bmi'].fillna(test['weight']/((test['height']/100)**2),inplace=True)

def weightedclasst(x):
    """based on : https://www.kaggle.com/c/widsdatathon2020/discussion/127987 but made to use ordinals"""
    if x < 15: return 0 
    elif x >= 15 and x < 16: return 1
    elif x >=16 and x < 18.5: return 2 
    elif x >= 18.5 and x < 25: return 3 
    elif x >= 25 and x < 30: return 4 
    elif x >= 30 and x < 35: return 5 
    elif x >= 35 and x < 40: return 6 
    else: return -1 
                                
df['weightclass'] = df['bmi'].map(weightedclasst)

In [11]:
imputer_skdf = train[['weight','height','bmi']]
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# Fit and transform to the parameters
imputer_skdf = pd.DataFrame(imputer.fit_transform(imputer_skdf))
imputer_skdf.columns = ['weight','height','bmi']
# Checking for any null values
imputer_skdf.isna().sum()
train['weight'] = imputer_skdf['weight']
train['height'] = imputer_skdf['height']
train['bmi'] =  imputer_skdf['bmi']
imputer_skdf = test[['weight','height','bmi']]
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# Fit and transform to the parameters
imputer_skdf = pd.DataFrame(imputer.fit_transform(imputer_skdf))
imputer_skdf.columns = ['weight','height','bmi']
# Checking for any null values
imputer_skdf.isna().sum()
test['weight'] = imputer_skdf['weight']
test['height'] = imputer_skdf['height']
test['bmi'] =  imputer_skdf['bmi']



In [14]:
cat_exploration('hepatic_failure')
print('min:',train['hepatic_failure'].min())
print('mean:',train['albumin_apache'].mean())
print('max',train['albumin_apache'].max())
print('med:',train['albumin_apache'].median())

min: 0.0
mean: 2.9029678041463725
max 4.6
med: 2.9


In [15]:
icu_id_intersect = set(train['icu_id']).intersection(test['icu_id'])
train['icu_id'].replace(to_replace=list(set(train['icu_id']) - icu_id_intersect),value=0,inplace=True)
test['icu_id'].replace(to_replace=list(set(test['icu_id']) - icu_id_intersect),value=0,inplace=True)

hospital_id_intersection = set(train['hospital_id']).intersection(test['hospital_id'])
train['hospital_id'].replace(to_replace=list(set(train['hospital_id']) - hospital_id_intersection),value=0,inplace=True)
test['hospital_id'].replace(to_replace=list(set(test['hospital_id']) - hospital_id_intersection),value=0,inplace=True)


In [18]:
# function to evaluate the score of our model
def eval_auc(pred,real):
    false_positive_rate, recall, thresholds = roc_curve(real, pred)
    roc_auc = auc(false_positive_rate, recall)
    return roc_auc    

In [19]:
# a wrapper class  that we can have the same ouput whatever the model we choose
class Base_Model(object):
    
    def __init__(self, train_df, test_df, features, categoricals=[], n_splits=5, verbose=True,ps={}):
        self.train_df = train_df
        self.test_df = test_df
        self.features = features
        self.n_splits = n_splits
        self.categoricals = categoricals
        self.target = 'hospital_death'
        self.cv = self.get_cv()
        self.verbose = verbose
#         self.params = self.get_params()
        self.params = self.set_params(ps)
        self.y_pred, self.score, self.model , self.oof_pred = self.fit()
        
    def train_model(self, train_set, val_set):
        raise NotImplementedError
        
    def get_cv(self):
        cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        return cv.split(self.train_df, self.train_df[self.target])
    
    def get_params(self):
        raise NotImplementedError
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        raise NotImplementedError
        
    def convert_x(self, x):
        return x
        
    def fit(self):
        oof_pred = np.zeros((len(self.train_df), ))
        y_pred = np.zeros((len(self.test_df), ))
        for fold, (train_idx, val_idx) in enumerate(self.cv):
            x_train, x_val = self.train_df[self.features].iloc[train_idx], self.train_df[self.features].iloc[val_idx]
            y_train, y_val = self.train_df[self.target][train_idx], self.train_df[self.target][val_idx]
            train_set, val_set = self.convert_dataset(x_train, y_train, x_val, y_val)
            model = self.train_model(train_set, val_set)
            conv_x_val = self.convert_x(x_val)
            oof_pred[val_idx] = model.predict(conv_x_val).reshape(oof_pred[val_idx].shape)
            x_test = self.convert_x(self.test_df[self.features])
            y_pred += model.predict(x_test).reshape(y_pred.shape) / self.n_splits

            print('Partial score of fold {} is: {}'.format(fold,eval_auc(oof_pred[val_idx],y_val) ))
        #print(oof_pred, self.train_df[self.target].values)
        loss_score = eval_auc(oof_pred,self.train_df[self.target].values) 
        if self.verbose:
            print('Our oof AUC score is: ', loss_score)
        return y_pred, loss_score, model , oof_pred

In [20]:
#we choose to try a LightGbM using the Base_Model class
class Lgb_Model(Base_Model):
    
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        return lgb.train(self.params, train_set, valid_sets=[train_set, val_set], verbose_eval=verbosity)
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=self.categoricals)
        val_set   = lgb.Dataset(x_val,    y_val,  categorical_feature=self.categoricals)
        return train_set, val_set
        
    def get_params(self):
        params = {'n_estimators':5000,
                    'boosting_type': 'gbdt',
                    'objective': 'binary',
                    'metric': 'auc',
                    'subsample': 0.75,
                    'subsample_freq': 1,
                    'learning_rate': 0.1,
                    'feature_fraction': 0.9,
                    'max_depth': 15,
                    'lambda_l1': 1,  
                    'lambda_l2': 1,
                    'early_stopping_rounds': 100,
                    #'is_unbalance' : True ,
                    'scale_pos_weight' : 3
                  
                    }
        return params
    def set_params(self,ps={}):
        params = self.get_params()
        if 'subsample_freq' in ps:
            params['subsample_freq']=int(ps['subsample_freq'])
            params['learning_rate']=ps['learning_rate']
            params['feature_fraction']=ps['feature_fraction']
            params['lambda_l1']=ps['lambda_l1']
            params['lambda_l2']=ps['lambda_l2']
            params['scale_pos_weight']=ps['scale_pos_weight']
            params['max_depth']=int(ps['max_depth'])
        
        return params  

In [24]:
# this is a list of features that look like to be categorical
categoricals_features = ['ethnicity','gender',
                         'hospital_admit_source','icu_admit_source','icu_stay_type',
                         'icu_type','apache_3j_bodysystem','apache_2_bodysystem', 'hospital_id'#,'icu_id'
                        #'apache_2_diagnosis','apache_3j_diagnosis'

                        ]
#new categorical features from imputation
categoricals_features += features_to_add_categorical

# this is the list of all input feature we would like our model to use (we remove the target and the ids.)
features = [col for col in train.columns if col not in ['encounter_id', 'patient_id',  'hospital_death']]
print('numeber of features ' , len(features))
print('shape of train / test ', train.shape , test.shape)

numeber of features  197
shape of train / test  (91713, 200) (39308, 200)


Feature Selection 

In [25]:
#feature selection added by Nurit
#consider not to drop icu_id
                                                     

X = train.drop('hospital_death',axis = 1)    
# Threshold for removing correlated variables
threshold = 0.99

# Absolute value correlation matrix
corr_matrix = X.corr().abs()
corr_matrix.head()
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()
# Select columns with correlations above threshold
corr_to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to remove.' % (len(corr_to_drop)))

features = [col for col in train.columns if col not in ['encounter_id', 'patient_id',  'hospital_death',
                                                        'readmission_status','icu_id'
                                                        #'paco2_apache'
                                                        #delete from imputation
                                                        
                                                       ]]
features = [col for col in features if col not in features_to_drop]
features = [col for col in features if col not in USELESS_COLS]
#Drop the columns with high correlations
# not improving results!!!
#features = [col for col in features if col not in corr_to_drop]
print('numeber of features ' , len(features))


There are 18 columns to remove.
numeber of features  181


categorical feature need to be transform to numeric for mathematical purpose.
different technics of categorical encoding exists here we will rely on our model API to deal with categorical
still we need to encode each categorical value to an id , for this purpose we use LabelEncoder


In [26]:
train.to_csv("train_fs.csv",index=False)
test.to_csv("train_fs.csv",index=False)

In [27]:
train

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,sodium_cat,wbc_cat,creatinine_cat,fio2_cat,temp_cat,albumin_cat,bilirubin_cat,map_cat,gcs_eyes_cat,gcs_motor_cat,gcs_verbal_cat,intubated_cat,ph_cat,glucose_cat
0,66154,25312,0,0,68.00,22.73,0,Caucasian,M,180.30,Floor,Floor,0,admit,CTICU,0.54,0,73.90,2.30,113.00,502.01,0,0.00,0.40,31.00,2.51,nan,3.00,6.00,0.00,4.00,168.00,118.00,27.40,0.00,40.00,42.16,40.00,132.06,nan,36.00,134.00,39.30,0.00,0.00,14.10,46.00,32.00,68.00,37.00,68.00,37.00,119.00,72.00,66.00,40.00,89.00,46.00,89.00,46.00,34.00,10.00,100.00,74.00,122.00,64.00,131.00,73.00,131.00,73.00,39.90,37.20,nan,nan,68.00,63.00,68.00,63.00,119.00,108.00,nan,nan,86.00,85.00,86.00,85.00,26.00,18.00,100.00,74.00,nan,nan,131.00,115.00,131.00,115.00,39.50,37.50,2.30,2.30,0.40,0.40,31.00,30.00,8.50,7.40,2.51,2.23,168.00,109.00,19.00,15.00,8.90,8.90,27.40,27.40,nan,nan,1.30,1.00,233.00,233.00,4.00,3.40,136.00,134.00,14.10,14.10,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na

In [26]:
# categorical feature need to be transform to numeric for mathematical purpose.
# different technics of categorical encoding exists here we will rely on our model API to deal with categorical
# still we need to encode each categorical value to an id , for this purpose we use LabelEncoder

print('Transform all String features to category.\n')
for usecol in categoricals_features:
    train[usecol] = train[usecol].astype('str')
    test[usecol] = test[usecol].astype('str')
    
    #Fit LabelEncoder
    le = LabelEncoder().fit(
            np.unique(train[usecol].unique().tolist()+
                      test[usecol].unique().tolist()))

    #At the end 0 will be used for dropped values
    train[usecol] = le.transform(train[usecol])+1
    test[usecol]  = le.transform(test[usecol])+1
    
    train[usecol] = train[usecol].replace(np.nan, 0).astype('int').astype('category')
    test[usecol]  = test[usecol].replace(np.nan, 0).astype('int').astype('category')

Transform all String features to category.



# Model

In [27]:
# percentage of death , hopefully it s a bit unbalanced
train['hospital_death'].sum()/train['hospital_death'].count()

0.08630183289173836

# hyper parameter tuning

In [30]:
def LGB_Beyes(subsample_freq,
                    learning_rate,
                    feature_fraction,
                    max_depth,
                    lambda_l1,
                    lambda_l2,
                    scale_pos_weight):
    params={}
    params['subsample_freq']=subsample_freq
    params['learning_rate']=learning_rate
    params['feature_fraction']=feature_fraction
    params['lambda_l1']=lambda_l1
    params['lambda_l2']=lambda_l2
    params['max_depth']=max_depth
    params['scale_pos_weight']=scale_pos_weight
    
    lgb_model= Lgb_Model(train, test, features, categoricals=categoricals_features,ps=params)
    print('auc: ',lgb_model.score)
    return lgb_model.score

bounds_LGB = {
    'subsample_freq': (1, 10),
    'learning_rate': (0.005, 0.02),
    'feature_fraction': (0.3, 1),
    'lambda_l1': (0, 5),
    'lambda_l2': (0, 5),
    'max_depth': (7, 17),
    'scale_pos_weight': (1, 10),
}

# ACTIVATE it if you want to search for better parameter
if 0 : 
    LGB_BO = BayesianOptimization(LGB_Beyes, bounds_LGB, random_state=1029)
    import warnings
    init_points = 16
    n_iter = 160
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')    
        LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

In [31]:
params = {'feature_fraction': 0.9,
 'lambda_l1': 1,
 'lambda_l2': 1,
 'learning_rate': 0.1,
 'max_depth': 13,
 'subsample_freq': 1,
 'scale_pos_weight':1}

In [ ]:
if 0: # ACTIVATE it if you want to search for better parameter
    lgb_model = Lgb_Model(train,test, features, categoricals=categoricals_features, ps= LGB_BO.max['params']  )
else :
    lgb_model = Lgb_Model(train,test, features, categoricals=categoricals_features, ps=params)

Feature Importance from the lightgbm model (gain)

In [ ]:
imp_df = pd.DataFrame()
imp_df['feature'] = features
imp_df['gain']  = lgb_model.model.feature_importance(importance_type='gain')
imp_df['split'] = lgb_model.model.feature_importance(importance_type='split')

In [ ]:
def plot_importances(importances_):
    mean_gain = importances_[['gain', 'feature']].groupby('feature').mean()
    importances_['mean_gain'] = importances_['feature'].map(mean_gain['gain'])
    plt.figure(figsize=(18, 44))
    data_imp = importances_.sort_values('mean_gain', ascending=False)
    sns.barplot(x='gain', y='feature', data=data_imp[:300])
    plt.tight_layout()
    plt.savefig('importances.png')
    plt.show()

In [ ]:
plot_importances(imp_df)

# Feature Importance by permutation importance algo

In [ ]:
import shap
explainer   =  shap.TreeExplainer(lgb_model.model)
shap_values = explainer.shap_values(train[features].iloc[:1000,:])
shap.summary_plot(shap_values, train[features].iloc[:1000,:])

# Some univariate plot of the best feature

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=UserWarning)
i=0
for index, row in imp_df.sort_values(by=['gain'],ascending=False).iterrows():  
    column=row['feature']
    i=i+1
    if i< 20:
            print(column,i,"gain :",row['gain'])
            df1      = train.loc[train['hospital_death']==0]
            df2      = train.loc[train['hospital_death']==1]

            fig = plt.figure(figsize=(20,4))
            sns.distplot(df1[column].dropna(),  color='red', label='hospital_death 0', kde=True); 
            sns.distplot(df2[column].dropna(),  color='blue', label='hospital_death 1', kde=True); 
            fig=plt.legend(loc='best')
            plt.xlabel(column, fontsize=12);
            plt.show()
            i=i+1


In [ ]:
test["hospital_death"] = lgb_model.y_pred
test[["encounter_id","hospital_death"]].to_csv("submission.csv",index=False)